In [1]:
import pandas
import duckdb
from src.club_names import name_dict

In [2]:
from os import walk

f = []
for (dirpath, dirnames, filenames) in walk("data/fifa/historical_data"):
    f.extend(filenames)
    

In [3]:
relevant_fields = ['club_name', 'overall', 'club_position', 'league_name']
dfs = []

for filename in f:
    path = "data/fifa/historical_data/" + filename
    new_df = pandas.read_csv(path,skipinitialspace=True, usecols=relevant_fields)
    new_df["season"] = filename[:-4]
    dfs.append(new_df)
    
players = pandas.concat(dfs)



In [4]:

results = pandas.read_csv(r"data/results_premier/results.csv", encoding = "ISO-8859-1")



In [5]:
query = """
    SELECT 
      club_name
      , season
      , avg(overall) filter(where club_position = 'RES') as reserve_overall
      , avg(overall) filter(where club_position = 'SUB') as subs_overall
      , avg(overall) filter(where club_position not in ('RES','SUB') ) as titular_overall
      , avg(overall) filter(where club_position in ('ST','CAM','LS','LW', 'RW','CF','RS' , 'LM' , 'RF') ) as attack_overall
      , avg(overall) filter(where club_position in ( 'GK', 'LCB', 'RCB' ,'CDM', 'LDM' , 'RDM', 'LB', 'RB' ,'CB', 'RCB' , 'LCB' ) ) as defend_overall

    FROM players 
    where league_name = 'English Premier League' 
    group by club_name, season
    
"""
teams = duckdb.query(query).to_df()


In [6]:

query = """
    SELECT 
        season ,
        HomeTeam,  
        AwayTeam,
        FTHG as goals_home,
        FTAG as goals_away
        
    FROM results
    where 
        season in (
            '2014-15',
            '2015-16',
            '2016-17', 
            '2017-18', 
            '2018-19',
            '2019-20',
            '2020-21'
            )
"""
results_21 = duckdb.query(query).to_df()

In [7]:
results_21["away_code"] = results_21.AwayTeam.apply(lambda x: name_dict[x])
results_21["home_code"] = results_21.HomeTeam.apply(lambda x: name_dict[x])

In [27]:
query = """
    SELECT 
        r.season,
        home_code as home
        ,away_code as away
        ,goals_home
        ,goals_away
        , th.reserve_overall as home_reserve_overall
        , th.subs_overall as home_subs_overall
        , th.titular_overall as home_titular_overall
        , th.attack_overall as home_attack_overall
        , th.defend_overall as home_defend_overall
        , ta.reserve_overall as away_reserve_overall
        , ta.subs_overall as away_subs_overall
        , ta.titular_overall as away_titular_overall
        , ta.attack_overall as away_attack_overall
        , ta.defend_overall as away_defend_overall
    FROM results_21 r
    Left Join teams th on (r.home_code, r.season) = (th.club_name, th.season)
    Left Join teams ta on (r.away_code, r.season) = (ta.club_name, ta.season)
"""
df = duckdb.query(query).to_df()

In [19]:
df.to_csv('data/ml/df.csv', index=False)

In [31]:
# Debugging : There are teams with data, but after the koin, empty values

,Season,home,away,goals_home,goals_away,home_reserve_overall,home_subs_overall,home_titular_overall,home_attack_overall,home_defend_overall,away_reserve_overall,away_subs_overall,away_titular_overall,away_attack_overall,away_defend_overall
0,2014-15,Manchester City,Chelsea,1,1,66.428571,78.909091,82.363636,85.000000,81.200000,62.000,77.090909,83.363636,86.333333,82.142857
1,2014-15,Tottenham Hotspur,West Bromwich Albion,0,1,60.500000,76.333333,78.200000,78.500000,78.000000,57.875,74.545455,74.636364,78.000000,73.666667
2,2014-15,Arsenal,Tottenham Hotspur,1,1,59.222222,79.166667,81.727273,84.000000,80.000000,60.500,76.333333,78.200000,78.500000,78.000000
3,2014-15,Chelsea,Aston Villa,3,0,62.000000,77.090909,83.363636,86.333333,82.142857,66.750,74.333333,75.200000,73.666667,76.166667
4,2014-15,Crystal Palace,Leicester City,2,0,66.333333,73.666667,75.454545,75.333333,75.285714,58.375,72.083333,74.454545,75.000000,73.600000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2655,2014-15,Queens Park Rangers,Sunderland,1,0,NaN,NaN,NaN,NaN,NaN,62.900,74.583333,75.500000,77.000000,74.571429
2656,2014-15,Hull City,West Ham United,2,2,NaN,NaN,NaN,NaN,NaN,59.500,75.500000,75.818182,77.333333,75.142857
2657,2014-15,Burnley,Sunderland,0,0,NaN,NaN,NaN,NaN,NaN,62.900,74.583333,75.500000,77.000000,74.571429
2658,2014-15,Queens Park Rangers,Stoke City,2,2,NaN,NaN,NaN,NaN,NaN,63.600,75.916667,76.272727,76.666667,75.571429


In [32]:
teams

,club_name,season,reserve_overall,subs_overall,titular_overall,attack_overall,defend_overall
0,Manchester City,2020-21,66.3,79.166667,85.818182,85.333333,85.000000
1,Liverpool,2020-21,70.5,77.166667,87.454545,89.000000,87.333333
2,Tottenham Hotspur,2020-21,68.7,79.333333,82.727273,85.666667,81.428571
3,Chelsea,2020-21,75.2,79.750000,82.272727,81.000000,82.428571
4,Arsenal,2020-21,72.3,75.250000,80.818182,84.000000,80.250000
...,...,...,...,...,...,...,...
135,AFC Bournemouth,2018-19,59.7,73.000000,76.181818,77.000000,77.000000
136,Newcastle United,2018-19,64.1,74.333333,76.545455,76.333333,76.600000
137,Brighton & Hove Albion,2018-19,64.9,74.083333,76.272727,77.000000,76.000000
138,Swansea City,2017-18,63.2,72.833333,76.818182,77.333333,77.333333
